# Aplica criação de variáveis com o método GAN (25%)

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

def apply_gan_to_generate_variables(file_path, percentage_generated_features=0.25, num_epochs=100, batch_size=32, learning_rate=0.001):
    print(f"Lendo arquivo de entrada: {file_path}...")
    data = pd.read_csv(file_path, delimiter=",", skipinitialspace=True)
    print("Arquivo carregado com sucesso.")

    print("Convertendo os dados para tensores do PyTorch...")
    data_tensor = torch.tensor(data.values, dtype=torch.float32)

    input_dim = data.shape[1]
    num_generated_features = max(1, int(input_dim * percentage_generated_features))  # Calcular a quantidade de variáveis a serem geradas
    print(f"A base de dados contém {input_dim} variáveis de entrada. Gerando {num_generated_features} novas variáveis.")

    class Generator(nn.Module):
        def __init__(self, input_dim, output_dim):
            super(Generator, self).__init__()
            self.model = nn.Sequential(
                nn.Linear(input_dim, 64),
                nn.ReLU(),
                nn.Linear(64, 64),
                nn.ReLU(),
                nn.Linear(64, output_dim)
            )
        
        def forward(self, x):
            return self.model(x)
    
    class Discriminator(nn.Module):
        def __init__(self, input_dim):
            super(Discriminator, self).__init__()
            self.model = nn.Sequential(
                nn.Linear(input_dim, 64),
                nn.LeakyReLU(0.2),
                nn.Linear(64, 64),
                nn.LeakyReLU(0.2),
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
        
        def forward(self, x):
            return self.model(x)
    
    print("Inicializando modelos Generator e Discriminator...")
    generator = Generator(input_dim, num_generated_features)
    discriminator = Discriminator(input_dim + num_generated_features)

    criterion = nn.BCELoss()
    optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate)
    optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

    dataset = TensorDataset(data_tensor)
    data_loader = DataLoader(dataset, batch_size=min(batch_size, len(dataset)), shuffle=True)

    print("Iniciando o treinamento da GAN...")
    for epoch in range(num_epochs):
        for batch in data_loader:
            real_data = batch[0]
            batch_size_actual = real_data.size(0)

            # Labels reais e falsas
            real_labels = torch.ones((batch_size_actual, 1))
            fake_labels = torch.zeros((batch_size_actual, 1))

            # Treinar o discriminador
            generated_data = generator(real_data)
            fake_data = torch.cat((real_data, generated_data), dim=1)
            real_data_extended = torch.cat((real_data, torch.zeros((batch_size_actual, num_generated_features))), dim=1)
            
            discriminator_real_loss = criterion(discriminator(real_data_extended), real_labels)
            discriminator_fake_loss = criterion(discriminator(fake_data.detach()), fake_labels)
            discriminator_loss = discriminator_real_loss + discriminator_fake_loss

            optimizer_D.zero_grad()
            discriminator_loss.backward()
            optimizer_D.step()

            # Treinar o gerador
            fake_data = torch.cat((real_data, generator(real_data)), dim=1)
            generator_loss = criterion(discriminator(fake_data), real_labels)

            optimizer_G.zero_grad()
            generator_loss.backward()
            optimizer_G.step()

        if (epoch + 1) % 10 == 0:
            print(f"Época [{epoch+1}/{num_epochs}] - Perda Discriminador: {discriminator_loss.item():.4f}, Perda Gerador: {generator_loss.item():.4f}")

    print("Treinamento concluído. Gerando novas variáveis...")
    with torch.no_grad():
        new_variables = generator(data_tensor).numpy()

    for i in range(num_generated_features):
        data[f"Generated_Var_{i+1}"] = new_variables[:, i]

    # Reorganizar as colunas para garantir que a coluna alvo seja a última
    target_column = data.columns[-(num_generated_features + 1)]  # Identifica a coluna alvo original
    columns = [col for col in data.columns if col != target_column] + [target_column]
    data = data[columns]

    print("Novas variáveis adicionadas e colunas reorganizadas com sucesso.")
    return data

def process_and_save_files(input_dir, output_dir, percentage_generated_features=0.25, num_epochs=100, batch_size=32, learning_rate=0.001):
    print(f"Verificando diretório de saída: {output_dir}...")
    os.makedirs(output_dir, exist_ok=True)
    print("Diretório de saída pronto.")

    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            
            print(f"\nProcessando arquivo: {input_path}")
            try:
                processed_data = apply_gan_to_generate_variables(
                    file_path=input_path,
                    percentage_generated_features=percentage_generated_features,
                    num_epochs=num_epochs,
                    batch_size=batch_size,
                    learning_rate=learning_rate
                )
                processed_data.to_csv(output_path, sep=",", index=False)
                print(f"Arquivo processado e salvo em: {output_path}")
            except Exception as e:
                print(f"Erro ao processar {filename}: {e}")

# Diretórios de entrada e saída
input_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\1-saida"
output_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\2-saida"

# Processar arquivos
process_and_save_files(input_directory, output_directory, percentage_generated_features=0.25, num_epochs=100)


Verificando diretório de saída: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\2-saida...
Diretório de saída pronto.

Processando arquivo: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\1-saida\2-saida_cocomo81.txt
Lendo arquivo de entrada: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\1-saida\2-saida_cocomo81.txt...
Arquivo carregado com sucesso.
Convertendo os dados para tensores do PyTorch...
A base de dados contém 17 variáveis de entrada. Gerando 4 novas variáveis.
Inicializando modelos Generator e Discriminator...
Iniciando o treinamento da GAN...
Época [10/100] - Perda Discriminador: 1.3912, Perda Gerador: 0.6887
Época [20/100] - Perda Discriminador: 1.3973, Perda Gerador: 0.6958
Época [30/100] - Perda Discriminador: 1.3809, Perda Gerador: 0.7242
Época [40/100] - Perda Discri